In [1]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig

pb = Predibase(api_token="...")
# https://app.predibase.com/

TypeError: ForwardRef.__init__() got an unexpected keyword argument 'is_class'

In [ ]:
# # dataset upload
# dataset = pb.datasets.from_file("./dataset/dataset.jsonl", name="medicine_dataset")

# using exist dataset
dataset = pb.datasets.get('medicine_dataset')

In [ ]:
# Create an adapter repository
repo = pb.repos.create(name="medicine_suggest-model", description="medicine suggetion chatbot", exists_ok=True)

# Start a fine-tuning job, blocks until training is finished
adapter = pb.adapters.create(
    config=FinetuningConfig(
        base_model="solar-1-mini-chat-240612"
    ),
    dataset=dataset, # Also accepts the dataset name as a string
    repo=repo,
    description="initial model with defaults"
)

In [ ]:
# Get adapter, blocking call if training is still in progress
adapter = pb.adapters.get("medicine_suggest-model/1")
adapter